In [8]:
import random

In [166]:
def get_pairs(hand, s):
    pairs = []
    for i in range(1, 14):
        if hand.count((s, i)) == 2:
            pairs.append(i)
    return pairs

def play_friend_A_Pair(hand, s):
    pairs = get_pairs(hand, s)
    if 10 in pairs or 13 in pairs:
        return 20
    if 5 in pairs:
        return 10
    if pairs:
        return 0
    this_suit = [x[1] for x in hand if x[0]==s]
    if not this_suit:
        return 20
    points = []
    if 10 in this_suit:
        points.append(10)
    if 13 in this_suit:
        points.append(10)
    if 5 in this_suit:
        points.append(5)
    if len(this_suit) == 1:
        points.append(10)
    return sum(sorted(points, reverse=True)[:2])

def play_friend_A_Singles(hand, s):
    this_suit = [x[1] for x in hand if x[0]==s]
    if not this_suit:
        return 20
    points = [(10 if c==13 else c) for c in this_suit if c in [5,10,13]]
    if len(this_suit) == 1:
        points.append(10)
    return sum(sorted(points, reverse=True)[:2])

def play_enemy_A_Pair(hand, s):
    pairs = get_pairs(hand, s)
    if [x for x in pairs if x not in [5,10,13]]:
        return 0
    if 5 in pairs:
        return 10
    if 10 in pairs or 13 in pairs:
        return 20
    this_suit = [x[1] for x in hand if x[0]==s]
    not_points = [x for x in this_suit if x not in [5,10,13]]
    if len(not_points) > 1:
        return 0
    num_left = min(2, len(this_suit)) - len(not_points)
    points = sorted([(10 if x==13 else x) for x in this_suit if x in [5,10,13]])
    return sum(points[:num_left])

def play_enemy_A_Singles(hand, s):
    this_suit = [x[1] for x in hand if x[0]==s]
    if len(this_suit) < 2:
        return -10
    not_points = [x for x in this_suit if x not in [5,10,13]]
    if len(not_points) > 1:
        return 0
    num_left = 2 - len(not_points)
    points = sorted([(10 if x==13 else x) for x in this_suit if x in [5,10,13]])
    return sum(points[:num_left])


In [12]:
cards = [(0,0), (0,1)]
for i in range(1, 5):
    cards += [(i, n) for n in range(1, 14)]
cards = cards * 2

In [ ]:
random.shuffle(cards)
hands = [sorted(cards[i*25:(i+1)*25]) for i in range(4)]

In [201]:
total = 0
nround = 0
nfew = 0
nbad = 0
ngood = 0
s = 1
p = 0
f = (p + 2) % 4
points_pair = 0
points_singles = 0

while nround < 200000:
    total += 1
    random.shuffle(cards)
    hands = [[x for x in sorted(cards[i*25:(i+1)*25]) if x[1]!=2] for i in range(4)]
    if hands[p].count((s, 1)) == 2:
        #print([x for x in hands[p] if x[0]==s])
        nround += 1
        pf2 = play_friend_A_Pair(hands[f], s)
        pf = play_friend_A_Singles(hands[f], s)
        points_pair += pf2
        points_singles += pf
        if pf2 < pf:
            nbad += 1
        #print([x for x in hands[f] if x[0]==s])
        #print('pair: {}, singles: {}'.format(pf2, pf))
        for e in [i for i in range(4) if i!=p and i!=f]:
            pf2 = play_enemy_A_Pair(hands[e], s)
            pf = play_enemy_A_Singles(hands[e], s)
            points_pair += pf2
            points_singles += pf
            if pf < 0 or pf2 < 0:
                nfew += 1
            if pf2 > pf:
                ngood += 1
            #print([x for x in hands[e] if x[0]==s])
            #print('pair: {}, singles: {}'.format(pf2, pf))


In [202]:
print('avg pair: {:.1f}, avg singles: {:.1f}, %few: {:.3f}, %bad: {:.3f}, %good: {:.3f}, %hit: {:.3f}'.format(
    points_pair/nround, points_singles/nround, nfew/nround, nbad/nround, ngood/nround, nround/total))


avg pair: 11.9, avg singles: 11.5, %few: 0.025, %bad: 0.266, %good: 0.225, %hit: 0.052
